In [ ]:
# Copyright 2024 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Serving Llama with Text Generation Inference (TGI) on Vertex AI
> [**Deploy Meta Llama 3.1 405B on Google Cloud Vertex AI** ](https://github.com/huggingface/blog/blob/main/llama31-on-vertex-ai.md)

> [**Text Generation Inference (TGI)**](https://github.com/huggingface/text-generation-inference) is a toolkit developed by Hugging Face for deploying and serving LLMs, with high performance text generation.

> [**Hugging Face DLCs**](https://github.com/huggingface/Google-Cloud-Containers) are pre-built and optimized Deep Learning Containers (DLCs) maintained by Hugging Face and Google Cloud teams to simplify environment configuration for your ML workloads.

In [1]:
# @title Install Vertex AI SDK and other required packages
%pip install --upgrade --user --quiet google-cloud-aiplatform huggingface_hub

In [16]:
# @title Define project information
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [17]:
# @title GCP Authentication

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)


In [18]:
# @title Authenticate your Hugging Face account
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) y


In [23]:
# @title Check Project ID and Location
PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

PROJECT_ID, LOCATION

('ai-hangsik', 'us-central1')

## Requirements

You will need to have the following IAM roles set:

- Artifact Registry Reader (roles/artifactregistry.reader)
- Vertex AI User (roles/aiplatform.user)

For more information about granting roles, see [Manage access](https://cloud.google.com/iam/docs/granting-changing-revoking-access).

---

You will also need to enable the following APIs (if not enabled already):

- Vertex AI API (aiplatform.googleapis.com)
- Artifact Registry API (artifactregistry.googleapis.com)

For more information about API enablement, see [Enabling APIs](https://cloud.google.com/apis/docs/getting-started#enabling_apis).

---

To access Llama on Hugging Face, you’re required to review and agree to the model usage license on the Hugging Face Hub for any of the models from the [meta-llama/Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct), and the access request will be processed inmediately.

In [ ]:
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")

SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member= $SERVICE_ACCOUNT \
    --role="roles/artifactregistry.reader"

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member= $SERVICE_ACCOUNT \
    --role="roles/aiplatform.user"

In [ ]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable artifactregistry.googleapis.com

## Register Google Gemma on Vertex AI

To serve Gemma with Text Generation Inference (TGI) on Vertex AI, you start importing the model on Vertex AI Model Registry.

The Vertex AI Model Registry is a central repository where you can manage the lifecycle of your ML models. From the Model Registry, you have an overview of your models so you can better organize, track, and train new versions. When you have a model version you would like to deploy, you can assign it to an endpoint directly from the registry, or using aliases, deploy models to an endpoint. The models on the Vertex AI Model Registry just contain the model configuration and not the weights per se, as it's not a storage.

Before going into the code to upload or import a model on Vertex AI, let's quickly review the arguments provided to the `aiplatform.Model.upload` method:

* **`display_name`** is the name that will be shown in the Vertex AI Model Registry.

* **`serving_container_image_uri`** is the location of the Hugging Face DLC for TGI that will be used for serving the model.

* **`serving_container_environment_variables`** are the environment variables that will be used during the container runtime, so these are aligned with the environment variables defined by `text-generation-inference`, which are analog to the [`text-generation-launcher` arguments](https://huggingface.co/docs/text-generation-inference/en/basic_tutorials/launcher). Additionally, the Hugging Face DLCs for TGI also capture the `AIP_` environment variables from Vertex AI as in [Vertex AI Documentation - Custom container requirements for prediction](https://cloud.google.com/vertex-ai/docs/predictions/custom-container-requirements).

    * `MODEL_ID` is the identifier of the model in the Hugging Face Hub. To explore all the supported models you can check [the Hugging Face Hub](https://huggingface.co/models?sort=trending&other=text-generation-inference).
    * `NUM_SHARD` is the number of shards to use if you don't want to use all GPUs on a given machine e.g. if you have two GPUs but you just want to use one for TGI then `NUM_SHARD=1`, otherwise it matches the `CUDA_VISIBLE_DEVICES`.
    * `MAX_INPUT_TOKENS` is the maximum allowed input length (expressed in number of tokens), the larger it is, the larger the prompt can be, but also more memory will be consumed.
    * `MAX_TOTAL_TOKENS` is the most important value to set as it defines the "memory budget" of running clients requests, the larger this value, the larger amount each request will be in your RAM and the less effective batching can be.
    * `MAX_BATCH_PREFILL_TOKENS` limits the number of tokens for the prefill operation, as it takes the most memory and is compute bound, it is interesting to limit the number of requests that can be sent.
    * `HUGGING_FACE_HUB_TOKEN` is the Hugging Face Hub token, required as [`google/gemma-7b-it`](https://huggingface.co/google/gemma-7b-it) is a gated model.

* (optional) **`serving_container_ports`** is the port where the Vertex AI endpoint |will be exposed, by default 8080.

For more information on the supported `aiplatform.Model.upload` arguments, check [its Python reference](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_upload).

In [24]:
from huggingface_hub import get_token

model = aiplatform.Model.upload(
    display_name="Llama-3.1-8B-Instruct",
    serving_container_image_uri="us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu121.2-2.ubuntu2204.py310",
    serving_container_environment_variables={
        "MODEL_ID": "meta-llama/Llama-3.1-8B-Instruct",
        "NUM_SHARD": "1",
        "MAX_INPUT_TOKENS": "512",
        "MAX_TOTAL_TOKENS": "1024",
        "MAX_BATCH_PREFILL_TOKENS": "1512",
        "HUGGING_FACE_HUB_TOKEN": get_token(),
    },
    serving_container_ports=[8080],
)
model.wait()

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/721521243942/locations/us-central1/models/704449403334688768/operations/3795330417561698304
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/721521243942/locations/us-central1/models/704449403334688768@1
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/721521243942/locations/us-central1/models/704449403334688768@1')


## Deploy Google Gemma on Vertex AI

After the model is registered on Vertex AI, you can deploy the model to an endpoint.

You need to first deploy a model to an endpoint before that model can be used to serve online predictions. Deploying a model associates physical resources with the model so it can serve online predictions with low latency.

Before going into the code to deploy a model to an endpoint, let's quickly review the arguments provided to the `aiplatform.Model.deploy` method:

- **`endpoint`** is the endpoint to deploy the model to, which is optional, and by default will be set to the model display name with the `_endpoint` suffix.
- **`machine_type`**, **`accelerator_type`** and **`accelerator_count`** are arguments that define which instance to use, and additionally, the accelerator to use and the number of accelerators, respectively. The `machine_type` and the `accelerator_type` are tied together, so you will need to select an instance that supports the accelerator that you are using and vice-versa. More information about the different instances at [Compute Engine Documentation - GPU machine types](https://cloud.google.com/compute/docs/gpus), and about the `accelerator_type` naming at [Vertex AI Documentation - MachineSpec](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/MachineSpec).

For more information on the supported `aiplatform.Model.deploy` arguments, you can check [its Python reference](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model#google_cloud_aiplatform_Model_deploy).

In [25]:
deployed_model = model.deploy(
    endpoint=aiplatform.Endpoint.create(display_name="Llama-3.1-8B-Instruct"),
    machine_type="g2-standard-4",
    accelerator_type="NVIDIA_L4",
    accelerator_count=1,
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/721521243942/locations/us-central1/endpoints/9019277389972111360/operations/3311193457619369984
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/721521243942/locations/us-central1/endpoints/9019277389972111360
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/721521243942/locations/us-central1/endpoints/9019277389972111360')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/721521243942/locations/us-central1/endpoints/9019277389972111360
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/721521243942/locations/us-central1/endpoints/9019277389972111360/operations/7182037337344311296
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/72152124394

> Note that the model deployment on Vertex AI can take around 15 to 25 minutes; most of the time being the allocation / reservation of the resources, setting up the network and security, and such.

## Online predictions on Vertex AI

Once the model is deployed on Vertex AI, you can run the online predictions using the `aiplatform.Endpoint.predict` method, which will send the requests to the running endpoint in the `/predict` route specified within the container following Vertex AI I/O payload formatting.

As you are serving a `text-generation` model fine-tuned for instruction-following, you will need to make sure that the chat template, if any, is applied correctly to the input conversation; meaning that `transformers` need to be installed so as to instantiate the `tokenizer` for [`google/gemma-7b-it`](https://huggingface.co/google/gemma-7b-it) and run the `apply_chat_template` method over the input conversation before sending the input within the payload to the Vertex AI endpoint.

> Note that the Messages API will be supported on Vertex AI on upcoming TGI releases, starting on 2.3, meaning that at the time of writing this post, the prompts need to be formatted before sending the request if you want to achieve nice results (assuming you are using an intruction-following model and not a base model).

In [9]:
%pip install --upgrade --user --quiet transformers jinja2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 2.7 MB/s eta 0:00:00


After the installation is complete, the following snippet will apply the chat template to the conversation:

In [31]:
import os
from huggingface_hub import get_token
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    token=get_token(),
)

messages = [
    {"role": "system", "content": "You are an assistant that responds as an AI expert."},
    {"role": "user", "content": "What's Deep learning?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)


In [32]:
output = deployed_model.predict(
    instances=[
        {
            "inputs": inputs,
            "parameters": {
                "max_new_tokens": 128,
                "do_sample": True,
                "top_p": 0.95,
                "temperature": 0.7,
            },
        },
    ]
)

output


Prediction(predictions=['Deep learning is a subfield of machine learning (ML) that specializes in artificial neural networks with multiple layers. These networks are modeled after the human brain\'s neural structures and are capable of learning complex patterns in data.\n\nIn a traditional neural network, the data flows through a series of interconnected nodes or "neurons," which perform computations to transform the input data into a predicted output. The architecture of deep neural networks involves numerous hidden layers of these nodes, with each layer learning representations of the data from the previous layer.\n\nDeep learning models are particularly useful for image and speech recognition tasks. Some of the key characteristics of deep learning include:\n\n1.'], deployed_model_id='6301157102761017344', metadata=None, model_version_id='1', model_resource_name='projects/721521243942/locations/us-central1/models/704449403334688768', explanations=None)

Which is what you will be sending within the payload to the deployed Vertex AI Endpoint, as well as [the generation parameters](https://huggingface.co/docs/huggingface_hub/main/en/package_reference/inference_client#huggingface_hub.InferenceClient.text_generation).

#### From a different session

To run the online prediction from a different session, you can run the following snippet.

In [35]:
import os

from google.cloud import aiplatform


aiplatform.init(project=PROJECT_ID, location=LOCATION)

endpoint_display_name = (
    "Llama-3.1-8B-Instruct"  # TODO: change to your endpoint display name
)


# Iterates over all the Vertex AI Endpoints within the current project and keeps the first match (if any), otherwise set to None
ENDPOINT_ID = next(
    (
        endpoint.name
        for endpoint in aiplatform.Endpoint.list()
        if endpoint.display_name == endpoint_display_name
    ),
    None,
)
assert ENDPOINT_ID, (
    "`ENDPOINT_ID` is not set, please make sure that the `endpoint_display_name` is correct at "
    f"https://console.cloud.google.com/vertex-ai/online-prediction/endpoints?project={os.getenv('PROJECT_ID')}"
)

endpoint = aiplatform.Endpoint(
    f"projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}"
)
output = endpoint.predict(
    instances=[
        {
            "inputs": "<bos><start_of_turn>user\nWhat's Deep Learning? in details<end_of_turn>\n<start_of_turn>model\n",
            "parameters": {
                "max_new_tokens": 128,
                "do_sample": True,
                "top_p": 0.95,
                "temperature": 0.7,
            },
        },
    ],
)
output

Prediction(predictions=["Deep learning is a subset of machine learning that utilizes neural networks to analyze and learn from data. In essence, it's a type of artificial intelligence (AI) that is modeled after the human brain's functions of vision, speech, pattern recognition, and decision-making.\n\nHere's a more detailed explanation:\n\n1. **Neural Networks**: Neural networks are the base of deep learning. They consist of layers of interconnected nodes (neurons) that process and transmit information. Each node receives input from the previous layer, performs a computation, and then sends the output to the next layer.\n2. **Hierarchical Representation**: Deep learning models consist of"], deployed_model_id='6301157102761017344', metadata=None, model_version_id='1', model_resource_name='projects/721521243942/locations/us-central1/models/704449403334688768', explanations=None)

### Via gcloud

You can also send the requests using the `gcloud` CLI via the `gcloud ai endpoints` command.

> Note that, before proceeding, you should either replace the values or set the following environment variables in advance from the Python variables set in the example, as follows:
>
> ```python
> import os
> os.environ["PROJECT_ID"] = PROJECT_ID
> os.environ["LOCATION"] = LOCATION
> os.environ["ENDPOINT_NAME"] = "google--gemma-7b-it-endpoint"
> ```

In [ ]:
%%bash
ENDPOINT_ID=$(gcloud ai endpoints list \
  --project=$PROJECT_ID \
  --region=$LOCATION \
  --filter="display_name=$ENDPOINT_NAME" \
  --format="value(name)" \
  | cut -d'/' -f6)

echo '{
  "instances": [
    {
      "inputs": "<bos><start_of_turn>user\nWhat'\''s Deep Learning?<end_of_turn>\n<start_of_turn>model\n",
      "parameters": {
        "max_new_tokens": 20,
        "do_sample": true,
        "top_p": 0.95,
        "temperature": 1.0
      }
    }
  ]
}' | gcloud ai endpoints predict $ENDPOINT_ID \
  --project=$PROJECT_ID \
  --region=$LOCATION \
  --json-request="-"

### Via cURL

Alternatively, you can also send the requests via `cURL`.

> Note that, before proceeding, you should either replace the values or set the following environment variables in advance from the Python variables set in the example, as follows:
>
> ```python
> import os
> os.environ["PROJECT_ID"] = PROJECT_ID
> os.environ["LOCATION"] = LOCATION
> os.environ["ENDPOINT_NAME"] = "google--gemma-7b-it-endpoint"
> ```

In [ ]:
%%bash
ENDPOINT_ID=$(gcloud ai endpoints list \
  --project=$PROJECT_ID \
  --region=$LOCATION \
  --filter="display_name=$ENDPOINT_NAME" \
  --format="value(name)" \
  | cut -d'/' -f6)

curl -X POST \
    -H "Authorization: Bearer $(gcloud auth print-access-token)" \
    -H "Content-Type: application/json" \
    https://$LOCATION-aiplatform.googleapis.com/v1/projects/$PROJECT_ID/locations/$LOCATION/endpoints/$ENDPOINT_ID:predict \
    -d '{
        "instances": [
            {
                "inputs": "<bos><start_of_turn>user\nWhat'\''s Deep Learning?<end_of_turn>\n<start_of_turn>model\n",
                "parameters": {
                    "max_new_tokens": 20,
                    "do_sample": true,
                    "top_p": 0.95,
                    "temperature": 1.0
                }
            }
        ]
    }'

## Cleaning up

Finally, you can already release the resources that you've created as follows, to avoid unnecessary costs:

- `deployed_model.undeploy_all` to undeploy the model from all the endpoints.
- `deployed_model.delete` to delete the endpoint/s where the model was deployed gracefully, after the `undeploy_all` method.
- `model.delete` to delete the model from the registry.

In [ ]:
deployed_model.undeploy_all()
deployed_model.delete()
model.delete()

Alternatively, you can also remove those from the Google Cloud Console following the steps:

- Go to Vertex AI in Google Cloud
- Go to Deploy and use -> Online prediction
- Click on the endpoint and then on the deployed model/s to "Undeploy model from endpoint"
- Then go back to the endpoint list and remove the endpoint
- Finally, go to Deploy and use -> Model Registry, and remove the model

## References

- [GitHub Repository - Hugging Face DLCs for Google Cloud](https://github.com/huggingface/Google-Cloud-Containers): contains all the containers developed by the collaboration of both Hugging Face and Google Cloud teams; as well as a lot of examples on both training and inference, covering both CPU and GPU, as well support for most of the models within the Hugging Face Hub.
- [Google Cloud Documentation - Hugging Face DLCs](https://cloud.google.com/deep-learning-containers/docs/choosing-container#hugging-face): contains a table with the latest released Hugging Face DLCs on Google Cloud.
- [Google Artifact Registry - Hugging Face DLCs](https://console.cloud.google.com/artifacts/docker/deeplearning-platform-release/us/gcr.io): contains all the DLCs released by Google Cloud that can be used.
- [Hugging Face Documentation - Google Cloud](https://huggingface.co/docs/google-cloud): contains the official Hugging Face documentation for the Google Cloud DLCs.